<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/part_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
!pip install clickhouse_driver --quiet
from clickhouse_driver import Client
from google.colab import userdata
import ipywidgets as widgets

In [6]:
# connection requisites
host = 'clickhouse.lab.karpov.courses'
dbase = 'default'
port = 9000
user = 'student'
password = userdata.get('ch2020_pwd')

In [8]:
client = Client(host=host, port=port, user=user,
                password=password, database=dbase)

### Задание 4.1
Посмотрим на среднюю частоту ответа среди хозяев (f) и суперхозяев (t). <br>
•	host_response_rate – частота ответа <br>
•	host_is_superhost – является ли суперхозяином

In [14]:
sql = f"""
SELECT
  host_is_superhost,
  avg(host_response_rate) as avg_host_response_rate
FROM
(SELECT
  DISTINCT host_id,
  host_is_superhost,
  toUInt8OrNull(replaceAll(host_response_rate, '%', '')) AS host_response_rate
FROM
  listings)
GROUP BY host_is_superhost
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,host_is_superhost,avg_host_response_rate
0,,NaN
1,t,98.000000
2,f,89.398451


### Задание 4.2

Сгруппируйте данные из listings по хозяевам (host_id) и посчитайте, какую цену за ночь в среднем каждый из них устанавливает (у одного хоста может быть несколько объявлений). Идентификаторы сдаваемого жилья объедините в отдельный массив. Таблицу отсортируйте по убыванию средней цены и убыванию host_id (в таком порядке). В качестве ответа укажите первый массив в результирующей таблице, состоящий более чем из двух id.

In [15]:
sql = f"""
SELECT
  host_id,
  AVG(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as avg_host_price,
  groupArray(id) as ids
FROM
  listings
GROUP BY
  host_id
HAVING length(ids) > 2
ORDER BY avg_host_price DESC, host_id DESC
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,host_id,avg_host_price,ids
0,187655517,6000.0,"[25757977, 25759146, 25802565, 25802651, 25802..."


### Задание 4.3
Немного усложним предыдущую задачу, и посчитаем разницу между максимальной и минимальной установленной ценой у каждого хозяина. В качестве ответа укажите идентификатор хоста, у которого разница оказалась наибольшей.

In [18]:
sql = f"""
SELECT
  host_id,
  max(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) -  min(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS price_diff
FROM
  listings
GROUP BY
  host_id
ORDER BY price_diff DESC
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,host_id,price_diff
0,155140624,7977.0


### Задание 4.4
Теперь сгруппируйте данные по типу жилья и выведите средние значения цены за ночь, размера депозита и цены уборки.

In [17]:
sql = f"""
SELECT
  room_type,
  avg(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) AS avg_price,
  avg(toFloat32OrNull(replaceRegexpAll(security_deposit , '[$,]', ''))) AS avg_security_deposit,
  avg(toFloat32OrNull(replaceRegexpAll(cleaning_fee , '[$,]', ''))) AS avg_cleaning_fee
FROM
  listings
GROUP BY
  room_type
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,room_type,avg_price,avg_security_deposit,avg_cleaning_fee
0,Private room,52.479105,117.249491,15.385309
1,Shared room,51.564189,103.614035,9.188172
2,Entire home/apt,83.348909,274.919989,37.015750


### Задание 4.5
Отлично! В каких частях города средняя стоимость за ночь является наиболее низкой?
Сгруппируйте данные по neighbourhood_cleansed и посчитайте среднюю цену за ночь в каждом районе. В качестве ответа введите название места, где средняя стоимость за ночь ниже всего.


In [19]:
sql = f"""
SELECT
  neighbourhood_cleansed,
  avg(toFloat32OrNull(replaceRegexpAll(price, '[$,]', ''))) as avg_price
FROM
  listings
GROUP BY neighbourhood_cleansed
ORDER BY avg_price
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,neighbourhood_cleansed,avg_price
0,Neu-Hohenschönhausen Süd,20.5


### Задание 4.6
В каких районах Берлина средняя площадь жилья, которое сдаётся целиком, является наибольшей? Отсортируйте по среднему и выберите топ-3.

In [20]:
sql = f"""
SELECT
  neighbourhood_cleansed,
  avg(toFloat32OrNull(square_feet)) as avg_square_feet
FROM
  listings
WHERE room_type = 'Entire home/apt'
GROUP BY neighbourhood_cleansed
ORDER BY avg_square_feet DESC
LIMIT 3
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,neighbourhood_cleansed,avg_square_feet
0,Kaulsdorf,2260.000000
1,Schöneberg-Süd,1752.666667
2,Lichtenrade,1237.500000


### Задание 4.7
Напоследок давайте посмотрим, какая из представленных комнат расположена ближе всего к центру города. В качестве ответа укажите id объявления.

In [21]:
sql = f"""
SELECT
  id,
  geoDistance(13.4050, 52.5200, toFloat64OrNull(longitude), toFloat64OrNull(latitude)) AS distance
FROM
  listings
WHERE
  room_type = 'Private room'
ORDER BY distance
LIMIT 1
"""
result, columns = client.execute(sql, with_column_types=True)
pd.DataFrame(result, columns=[tuple[0] for tuple in columns])

,id,distance
0,19765058,58.801819
